## A liga ternária mais cara do mundo


## Introdução



Delete este texto e escreva uma breve introdução sobre seu experimento.



## Objetivo



Encontre uma liga de três elementos que tenha o maior custo possível. A liga ternária deve ser da forma $x$A.$y$B.$z$C sendo que $x+y+z = 100\,\mathrm{g}$, $x>5\,\mathrm{g}$, $y>5\,\mathrm{g}$, $z>5\,\mathrm{g}$ e &ldquo;A&rdquo;, &ldquo;B&rdquo; e &ldquo;C&rdquo; são elementos químicos. Utilize o preço [1] dado abaixo. Considere que qualquer composto com 3 elementos químicos é chamado de liga.


<h3> Desenvolvimento do algortimo </h3>
<p aling="justify"> Para resolver este problema, é necessário pensar primeiramente no formato dos individuos que serão gerados e mutados. Seguindo as dicas dadas, o método desenvolvido trabalhará com listas referentes as caracteristicas dos genes/elementos, afim de simplificar a iteração. </p>
 
<h4> Parametros iniciais </h4>
<ul>
    <li>A liga deve pesar 100g </li>
    <li>Os pesos e valores são determinados por uma lista pré-existente (espaço de busca)</li>
    <li>A parcela de cada elemento na liga deve ser maior que 5%</li>
</ul>

<h4> Passos</h4>
<ul>
    <li> Selecionar genes, que serão elementos da lista ✔ (funcoes.py) </li>
    <li> Gerar individuo válido </li>
    <li> Gerar uma população </li>
    <li> Avaliar seu respectivo preço (fitness_ind e fitness_pop) </li>
    

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
"""
from funcoes import gene_liga as elem_liga # seleciona um gene aleatório 
from funcoes import individuo_liga as ind_liga # gerar o individuo 
from funcoes import populacao_inicial_liga as pop_liga # gera uma população incial 
from funcoes import funcao_obj_liga as funcao_fitness # gera um valor de fitness para o individou 
from funcoes import funcao_objetivo_pop_senha # para calcular o preço (fitness)
import itertools
"""

'\nfrom funcoes import gene_liga as elem_liga # seleciona um gene aleatório \nfrom funcoes import individuo_liga as ind_liga # gerar o individuo \nfrom funcoes import populacao_inicial_liga as pop_liga # gera uma população incial \nfrom funcoes import funcao_obj_liga as funcao_fitness # gera um valor de fitness para o individou \nfrom funcoes import funcao_objetivo_pop_senha # para calcular o preço (fitness)\nimport itertools\n'

## Códigos e discussão



-   Use células de código para o código.

-   Use células de texto para a discussão.

-   A discussão não deve ser feita em comentários dentro das células de código. Toda discussão deve acontecer após o resultado sendo discutido foi apresentado. Exemplo: não discuta um gráfico antes de apresentá-lo.



In [2]:
# preço em dólares por kilograma
preco_kg = {
    "H": 1.39,
    "He": 24,
    "Li": 85.6,
    "Be": 857,
    "B": 3.68,
    "C": 0.122,
    "N": 0.14,
    "O": 0.154,
    "F": 2.16,
    "Ne": 240,
    "Na": 3.43,
    "Mg": 2.32,
    "Al": 1.79,
    "Si": 1.7,
    "P": 2.69,
    "S": 0.0926,
    "Cl": 0.082,
    "Ar": 0.931,
    "K": 13.6,
    "Ca": 2.35,
    "Sc": 3460,
    "Ti": 11.7,
    "V": 385,
    "Cr": 9.4,
    "Mn": 1.82,
    "Fe": 0.424,
    "Co": 32.8,
    "Ni": 13.9,
    "Cu": 6,
    "Zn": 2.55,
    "Ga": 148,
    "Ge": 1010,
    "As": 1.31,
    "Se": 21.4,
    "Br": 4.39,
    "Kr": 290,
    "Rb": 15500,
    "Sr": 6.68,
    "Y": 31,
    "Nb": 85.6,
    "Mo": 40.1,
    "Tc": 100000,
    "Ru": 10600,
    "Rh": 147000,
    "Pd": 49500,
    "Ag": 521,
    "Cd": 2.73,
    "In": 167,
    "Sn": 18.7,
    "Sb": 5.79,
    "Te": 63.5,
    "I": 35,
    "Xe": 1800,
    "Cs": 61800,
    "Ba": 0.275,
    "La": 4.92,
    "Ce": 4.71,
    "Pr": 103,
    "Nd": 57.5,
    "Pm": 460000,
    "Sm": 13.9,
    "Eu": 31.4,
    "Gd": 28.6,
    "Tb": 658,
    "Dy": 307,
    "Ho": 57.1,
    "Er": 26.4,
    "Tm": 3000,
    "Yb": 17.1,
    "Lu": 643,
    "Hf": 900,
    "Ta": 312,
    "W": 35.3,
    "Re": 4150,
    "Os": 12000,
    "Ir": 56200,
    "Pt": 27800,
    "Hg": 30.2,
    "Tl": 4200,
    "Pb": 2,
    "Bi": 6.36,
    "Po": 49200000000000,
    "Ac": 29000000000000,
    "Th": 287,
    "Pa": 280000,
    "U": 101,
    "Np": 660000,
    "Pu": 6490000,
    "Am": 750000,
    "Cm": 160000000000,
    "Bk": 185000000000,
    "Cf": 185000000000,
}

# converter a lista para gramas

preco_g = {}

for i in preco_kg:
    a = preco_kg[i]*0.001
    preco_g[i] = a
    
#print(preco_g)

In [3]:
"""
# Constantes

# Referentes ao problema 
NUM_GENES = 3 
PESO_LIGA = 100
PESO_MIN_ELEM = 5

# Referentes a busca
TAMANHO_POP = 100
"""

'\n# Constantes\n\n# Referentes ao problema \nNUM_GENES = 3 \nPESO_LIGA = 100\nPESO_MIN_ELEM = 5\n\n# Referentes a busca\nTAMANHO_POP = 100\n'

## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [19]:
import random
import itertools
lista_teste = {"ana": 7,
               "thom": 17,
               "monyque": 10
              }
peso = {} 

def gene_liga(lista_elementos):
    
    """Sorteia um elememto da lista.
    
        Args:
          lista_elementos: lista de elementos
        
        Return:
          Retorna um elemento da lista e um valor númerico representando seu peso na liga 
    
    """
    lista_gene = []
    
    massa_elementos = []
    
    nomes_elementos = []
        
    for massa in range(5, 91):
        massa_elementos.append(massa)
        
    for elemento in lista_elementos.keys():
        nomes_elementos.append(elemento)
        
    
    for m in nomes_elementos:
        gene = itertools.product(nomes_elementos, random.choices(massa))
    
    
    
    return gene

print(gene_liga(lista_teste))

TypeError: object of type 'int' has no len()

In [ ]:
def individuo_liga(lista_elementos, num_elementos, peso_liga):
    ''' gera um individuo valido para a liga
        
        OBS. O individuo deve ter um peso igual a 100g no final e
        cada elemento precisa estar presente em pelo menos 5% da liga 
        
            Args: 
                lista_elementos: lista com os elementos
                num_elementos: número de elementos presentes na liga
                peso_liga: peso máximo que a liga pode assumir 
                
            Return:
                individuo valido 
                valor da liga 
    '''


    individuo = []
    
    peso = 0
                         
    for i in range(num_elementos):

        elemento = (gene_liga(lista_elementos))
        individuo.append(elemento)
        
    return individuo

In [ ]:
NUM_ELEMENTOS = 3
PESO_LIGA = 100
individuo_liga(lista_teste, NUM_ELEMENTOS, PESO_LIGA)

[[('ana', 80, 17)], [('ana', 45, 10)], [('monyque', 17, 10)]]